In [4]:
import pandas as pd
import hvplot.pandas

In [12]:
prices_df = pd.read_csv('data/public_goods_prices.csv')
prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'])

In [13]:
prices_df[prices_df['token_id']=='token-engineering-commons']

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices
4615,0.000514,0.0,0.0,2021-07-14,token-engineering-commons,1.000000
4616,0.000501,0.0,0.0,2021-07-15,token-engineering-commons,0.973323
4617,0.000523,0.0,0.0,2021-07-16,token-engineering-commons,1.017616
4618,0.000534,0.0,0.0,2021-07-17,token-engineering-commons,1.037449
4619,0.000526,0.0,0.0,2021-07-18,token-engineering-commons,1.023449
...,...,...,...,...,...,...
5339,0.000269,0.0,0.0,2023-07-08,token-engineering-commons,0.522438
5340,0.000266,0.0,0.0,2023-07-09,token-engineering-commons,0.516663
5341,0.000271,0.0,0.0,2023-07-10,token-engineering-commons,0.526094
5342,0.000265,0.0,0.0,2023-07-11,token-engineering-commons,0.516177


In [14]:
prices_df[prices_df['token_id']=='token-engineering-commons'].hvplot.step(x='timestamp', y='prices', by='token_id')

:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

In [16]:
prices_df['normalized_prices'] = prices_df['prices'] / prices_df.groupby('token_id')['prices'].transform('first')

In [17]:
prices_df.hvplot.step(x='timestamp', y='normalized_prices', by='token_id')

:NdOverlay   [token_id]
   :Curve   [timestamp]   (normalized_prices)

In [18]:
import holoviews as hv

prices_df_adjusted = prices_df.copy(deep=True)

prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'] = prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'].iloc[20:]

prices_df_adjusted['normalized_prices'] = prices_df_adjusted['prices'] / prices_df_adjusted.groupby('token_id')['prices'].transform('first')

In [19]:
import datetime

def ordinal(n):
    """Return number with ordinal suffix."""
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

def formatted_date(past=False):
    """Return today's date in the desired format."""
    today = datetime.date.today()
    month = today.strftime('%b')
    day = ordinal(today.day)
    year = today.year
    if past:
        year = year - 2
    return f"{month} {day} {year}"

print(formatted_date())

May 2nd 2024


In [20]:
today = formatted_date()
past = formatted_date(past=True)

In [21]:
plot = prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].hvplot.line(logy=False, legend='top_right', grid=True, rot=45, title=f'Public Goods Tokens Normalized against Ethereum {past} - {today}',x='timestamp', y='normalized_prices', by='token_id', width=1400, height=600, line_width=3) * hv.HLine(1)
plot

:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [22]:
plot.opts(xticks=[(date, date.strftime('%B %Y')) for date in pd.date_range(start=prices_df_adjusted['timestamp'].min(), end=prices_df_adjusted['timestamp'].max(), freq='MS')])

:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [23]:
prices_df_adjusted['returns'] = prices_df_adjusted.groupby('token_id')['normalized_prices'].transform(lambda x: x.bfill().pct_change()+1)

In [24]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.0,77.759766,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.0,77.128195,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.0,52.110867,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.0,73.638198,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.0,107.827147,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
5339,0.000269,0.0,0.000000,2023-07-08,token-engineering-commons,0.522438,0.992685
5340,0.000266,0.0,0.000000,2023-07-09,token-engineering-commons,0.516663,0.988946
5341,0.000271,0.0,0.000000,2023-07-10,token-engineering-commons,0.526094,1.018252
5342,0.000265,0.0,0.000000,2023-07-11,token-engineering-commons,0.516177,0.981151


In [25]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
token-engineering-commons,1.002480,0.123751,0.020039
giveth,0.999817,0.156049,-0.001172
metagame,0.998914,0.069461,-0.015637
gitcoin,0.998698,0.047053,-0.027671
bright-token,0.998276,0.046252,-0.037276
rad,0.995049,0.098520,-0.050250
honey-finance,0.997106,0.055895,-0.051777


In [26]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').apply(lambda x: x['normalized_prices'].iloc[-1] / x['returns'].std()).sort_values(ascending=False)

/tmp/ipykernel_116834/757977197.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').apply(lambda x: x['normalized_prices'].iloc[-1] / x['returns'].std()).sort_values(ascending=False)


token_id
token-engineering-commons    4.201569
gitcoin                      3.768361
metagame                     3.550135
bright-token                 2.796049
honey-finance                1.121110
giveth                       0.167774
rad                          0.022392
dtype: float64

In [27]:
prices_df_adjusted.groupby('token_id')['normalized_prices'].apply(lambda x: (x.iloc[-1] / x.std())).sort_values(ascending=False)

token_id
ethereum                     4.600092
token-engineering-commons    0.880956
metagame                     0.827969
gitcoin                      0.806406
bright-token                 0.539641
honey-finance                0.133491
giveth                       0.086642
rad                          0.007521
Name: normalized_prices, dtype: float64

In [28]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,returns
0,0.000115,0.0,77.759766,2022-01-23,bright-token,1.000000,NaN
1,0.000113,0.0,77.128195,2022-01-24,bright-token,0.986009,0.986009
2,0.000121,0.0,52.110867,2022-01-25,bright-token,1.055293,1.070267
3,0.000118,0.0,73.638198,2022-01-26,bright-token,1.025065,0.971356
4,0.000121,0.0,107.827147,2022-01-27,bright-token,1.055378,1.029572
...,...,...,...,...,...,...,...
5339,0.000269,0.0,0.000000,2023-07-08,token-engineering-commons,0.522438,0.992685
5340,0.000266,0.0,0.000000,2023-07-09,token-engineering-commons,0.516663,0.988946
5341,0.000271,0.0,0.000000,2023-07-10,token-engineering-commons,0.526094,1.018252
5342,0.000265,0.0,0.000000,2023-07-11,token-engineering-commons,0.516177,0.981151


In [29]:
prices_df_adjusted.reset_index()['returns'].bfill().idxmax()

1596

In [30]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last').sort_values(ascending=False).to_frame(name=f'Return {past} - {today}').reset_index()

,token_id,Return May 2nd 2022 - May 2nd 2024
0,token-engineering-commons,0.519949
1,metagame,0.246596
2,gitcoin,0.177311
3,bright-token,0.129324
4,honey-finance,0.062665
5,giveth,0.026181
6,rad,0.002206


In [31]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std').sort_values(ascending=True).to_frame(name=f'Volatility {past} - {today}').reset_index()

,token_id,Volatility May 2nd 2022 - May 2nd 2024
0,bright-token,0.046252
1,gitcoin,0.047053
2,honey-finance,0.055895
3,metagame,0.069461
4,rad,0.098520
5,token-engineering-commons,0.123751
6,giveth,0.156049


In [32]:
(prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last') / prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std')).sort_values(ascending=False).to_frame(name=f'Pseudo Sharpe {past} - {today}').reset_index()

,token_id,Pseudo Sharpe May 2nd 2022 - May 2nd 2024
0,token-engineering-commons,4.201569
1,gitcoin,3.768361
2,metagame,3.550135
3,bright-token,2.796049
4,honey-finance,1.121110
5,giveth,0.167774
6,rad,0.022392


In [33]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
token-engineering-commons,1.002480,0.123751,0.020039
giveth,0.999817,0.156049,-0.001172
metagame,0.998914,0.069461,-0.015637
gitcoin,0.998698,0.047053,-0.027671
bright-token,0.998276,0.046252,-0.037276
rad,0.995049,0.098520,-0.050250
honey-finance,0.997106,0.055895,-0.051777
